# SEC Edgar data prep



## Imports

In [1]:
from dotenv import load_dotenv
import os
import json
import textwrap
from tqdm import tqdm

from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings



## Set up Neo4j

In [2]:
# Load from environment
load_dotenv('../.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'


IMPORT_DATA_DIRECTORY = '../data/sample/'

if OPENAI_API_KEY is None:
  raise ValueError("OPENAI_API_KEY is not set. Please add it to your .env file.")


In [3]:
from neo4j import GraphDatabase

gdb = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD) )

def isNeo4jReady() -> bool:
  try:
    gdb.verify_connectivity()
    return True
  except:
      return False
  
def showNeo4jReady():
  if isNeo4jReady():
    print("Neo4j ready")
  else:
    print("Neo4j not ready")

def showIndexes():
  try:
    records, _, _ = gdb.execute_query("SHOW INDEXES")
    for record in records:
      print(f'{record["type"]} INDEX: {record["name"]} {record["labelsOrTypes"]} {record["properties"]}')
  except:
    print("FAILED")

def dropGraph():
    with gdb.session() as session:
      session.run("""
        MATCH (n)
        CALL (n) {
          DETACH DELETE n
        } IN TRANSACTIONS OF 10000 ROWS;
        """)
    
def dropIndexesAndConstraints():
    for constraint in gdb.execute_query('SHOW CONSTRAINTS').records:
      gdb.execute_query(f"DROP CONSTRAINT {constraint['name']}")
    for index in gdb.execute_query('SHOW INDEXES').records:
      gdb.execute_query(f"""DROP INDEX `{index['name']}`""")
    
def resetDatabase():
   dropGraph()
   dropIndexesAndConstraints();

showNeo4jReady()
showIndexes()

Neo4j ready
VECTOR INDEX: form_10k_chunks ['Chunk'] ['textEmbedding']
RANGE INDEX: unique_chunk ['Chunk'] ['chunkId']


In [4]:
# To reset the entire Neo4j database, uncomment the following line then run the cell.
# resetDatabase()

# WARNING: don't forget to re-comment the line to avoid accidental reset

In [5]:

# Create a knowledge graph using Langchain's Neo4j integration.
# This will be used for direct querying of the knowledge graph. 
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [6]:
# Splitting text into chunks using the RecursiveCharacterTextSplitter 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

In [7]:
# Create a vector index called "form_10k_chunks" the `textEmbedding`` property of nodes labeled `Chunk`. 
# neo4j_create_vector_index(kg, VECTOR_INDEX_NAME, 'Chunk', 'textEmbedding')
kg.query("""
         CREATE VECTOR INDEX `form_10k_chunks` IF NOT EXISTS
          FOR (n:Chunk) ON (n.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

# Create a uniqueness constraint on the chunkId property of Chunk nodes 
kg.query('CREATE CONSTRAINT unique_chunk IF NOT EXISTS FOR (n:Chunk) REQUIRE n.chunkId IS UNIQUE')

showIndexes()


VECTOR INDEX: form_10k_chunks ['Chunk'] ['textEmbedding']
RANGE INDEX: unique_chunk ['Chunk'] ['chunkId']


In [8]:
def split_form10k_data_from_file(file):
    chunks_with_metadata = [] # use this to accumulate chunk records
    file_as_object = json.load(open(file)) # open the json file
    for item in ['item1','item1a','item7','item7a']: # pull these keys from the json
        print(f'Processing {item} from {file}') 
        item_text = file_as_object[item] # grab the text of the item
        item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
        chunk_seq_id = 0
        for chunk in item_text_chunks[:20]: # only take the first 20 chunks
            form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
            # finally, construct a record with metadata and the chunk text
            chunks_with_metadata.append({
                'text': chunk, 
                # metadata from looping...
                'f10kItem': item,
                'chunkSeqId': chunk_seq_id,
                # constructed metadata...
                'formId': f'{form_id}', # pulled from the filename
                'chunkId': f'{form_id}-{item}-chunk{chunk_seq_id:04d}',
                # metadata from file...
                'names': file_as_object['names'],
                'cik': file_as_object['cik'],
                'cusip6': file_as_object['cusip6'],
                'source': file_as_object['source'],
            })
            chunk_seq_id += 1
        # print(f'\tSplit into {chunk_seq_id} chunks')
    return chunks_with_metadata

def create_nodes_for_all_chunks(chunks_with_metadata_list):
    merge_chunk_node_query = """
    MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
        ON CREATE SET 
            mergedChunk.names = $chunkParam.names,
            mergedChunk.formId = $chunkParam.formId, 
            mergedChunk.cik = $chunkParam.cik, 
            mergedChunk.cusip6 = $chunkParam.cusip6, 
            mergedChunk.source = $chunkParam.source, 
            mergedChunk.item = $chunkParam.f10kItem, 
            mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
            mergedChunk.text = $chunkParam.text
    RETURN mergedChunk
    """
    node_count = 0
    for chunk in tqdm(chunks_with_metadata_list):
        # print(f"Creating `:Chunk` node for chunk ID {chunk['chunkId']}")
        kg.query(merge_chunk_node_query, 
                params={
                    'chunkParam': chunk
                })
        node_count += 1
    print(f"Created {node_count} nodes")

In [9]:
%%time

import glob

form10kDir = os.path.join(IMPORT_DATA_DIRECTORY, "form10k")

all_json_files = glob.glob(os.path.join(form10kDir, "*.json"))

counter = 0

for file_name in all_json_files:
    counter += 1
    print(f'=== Processing {counter} of {len(all_json_files)} ===')
    # get and split text data
    print('Reading and splitting Form10k file...')
    chunk_list = split_form10k_data_from_file(file_name)
    #load nodes
    print('Creating Chunk Nodes...')
    create_nodes_for_all_chunks(chunk_list)
    print(f'Done Processing {file_name}')

# Check the number of nodes in the graph
kg.query("MATCH (n:Chunk) RETURN count(n) as chunkCount")

=== Processing 1 of 10 ===
Reading and splitting Form10k file...
Processing item1 from ../data/sample/form10k/0001650372-23-000040.json
Processing item1a from ../data/sample/form10k/0001650372-23-000040.json
Processing item7 from ../data/sample/form10k/0001650372-23-000040.json
Processing item7a from ../data/sample/form10k/0001650372-23-000040.json
Creating Chunk Nodes...


100%|██████████| 64/64 [00:01<00:00, 32.47it/s]


Created 64 nodes
Done Processing ../data/sample/form10k/0001650372-23-000040.json
=== Processing 2 of 10 ===
Reading and splitting Form10k file...
Processing item1 from ../data/sample/form10k/0000950170-23-033201.json
Processing item1a from ../data/sample/form10k/0000950170-23-033201.json
Processing item7 from ../data/sample/form10k/0000950170-23-033201.json
Processing item7a from ../data/sample/form10k/0000950170-23-033201.json
Creating Chunk Nodes...


100%|██████████| 63/63 [00:01<00:00, 41.17it/s]


Created 63 nodes
Done Processing ../data/sample/form10k/0000950170-23-033201.json
=== Processing 3 of 10 ===
Reading and splitting Form10k file...
Processing item1 from ../data/sample/form10k/0001564708-23-000368.json
Processing item1a from ../data/sample/form10k/0001564708-23-000368.json
Processing item7 from ../data/sample/form10k/0001564708-23-000368.json
Processing item7a from ../data/sample/form10k/0001564708-23-000368.json
Creating Chunk Nodes...


...

100%|██████████| 64/64 [00:01<00:00, 43.19it/s]


Created 64 nodes
Done Processing ../data/sample/form10k/0000320187-23-000039.json
=== Processing 10 of 10 ===
Reading and splitting Form10k file...
Processing item1 from ../data/sample/form10k/0001137789-23-000049.json
Processing item1a from ../data/sample/form10k/0001137789-23-000049.json
Processing item7 from ../data/sample/form10k/0001137789-23-000049.json
Processing item7a from ../data/sample/form10k/0001137789-23-000049.json
Creating Chunk Nodes...


100%|██████████| 64/64 [00:01<00:00, 43.24it/s]

Created 64 nodes
Done Processing ../data/sample/form10k/0001137789-23-000049.json
CPU times: user 772 ms, sys: 128 ms, total: 900 ms
Wall time: 13.5 s


[{'chunkCount': 544}]

In [10]:
# Create vector embeddings for Chunks
embeddings_api = embeddings_api = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
chat_api = ChatOpenAI()

print("Finding all chunks that need textEmbedding...")
all_chunk_text_id = gdb.execute_query("""
  MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
  RETURN chunk.text AS text, chunk.chunkId AS chunkId
  """).records

print("Generating vector embeddings, then writing into each chunk...")
for chunk_text_id in tqdm(all_chunk_text_id):
  text_embedding = embeddings_api.embed_query(chunk_text_id['text'])
  gdb.execute_query("""
    MATCH (chunk:Chunk {chunkId: $chunkIdParam})
    CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $textEmbeddingParam)    
    """, 
    chunkIdParam=chunk_text_id['chunkId'], textEmbeddingParam=text_embedding
  )


Finding all chunks that need textEmbedding...
Generating vector embeddings, then writing into each chunk...


100%|██████████| 544/544 [04:09<00:00,  2.18it/s]


In [14]:
# Connect chunks into linked-lists

# Collect all the form IDs and form 10k item names
distinct_form_id_result = gdb.execute_query("""
MATCH (c:Chunk) RETURN DISTINCT c.formId as formId
""").records

distinct_form_id_list = list(map(lambda x: x['formId'], distinct_form_id_result))

# Connect *all* section chunks into a linked list..
cypher = """
  MATCH (from_same_form_and_section:Chunk) // match all chunks
  WHERE from_same_form_and_section.formId = $formIdParam // where the chunks are from the same form
    AND from_same_form_and_section.item = $itemParam // and from the same section
  WITH from_same_form_and_section // with those collections of chunks
    ORDER BY from_same_form_and_section.chunkSeqId ASC // order the chunks by their sequence ID
  WITH collect(from_same_form_and_section) as section_chunk_list // collect the chunks into a list
    CALL apoc.nodes.link(section_chunk_list, "NEXT", {avoidDuplicates: true}) // then create a linked list in the graph
  RETURN size(section_chunk_list)
"""

for form_id in distinct_form_id_list:
  for form10kItemName in ['item1', 'item1a', 'item7', 'item7a']:
    gdb.execute_query(cypher, 
             formIdParam=form_id, itemParam=form10kItemName
    )

gdb.execute_query("MATCH p=()-[:NEXT]->() RETURN count(p) as pathCount").records[0]

<Record pathCount=504>

In [15]:
# Helper function for extracting metadata based from a form10k json file
def extract_form10k_form_from_file(file):
    file_as_object = json.load(open(file)) # open the json file
    form_id = file[file.rindex('/') + 1:file.rindex('.')] # extract form id from file name
    full_text = f"""About {file_as_object['names']}...
      {file_as_object['item1'] if 'item1' in file_as_object else ''}
      {file_as_object['item1a'] if 'item1a' in file_as_object else ''}
      {file_as_object['item7'] if 'item7' in file_as_object else ''}
      {file_as_object['item7a'] if 'item7a' in file_as_object else ''}
      """

    form_with_metadata = {
      'formId': f'{form_id}', # pulled from the filename
      # metadata from file...
      'names': file_as_object['names'],
      'cik': file_as_object['cik'],
      'cusip6': file_as_object['cusip6'],
      'source': file_as_object['source'],
      'fullText': full_text
    }
    
    return form_with_metadata

In [16]:
# Create a uniqueness constraint for Form nodes (one per 10-k document)
gdb.execute_query('CREATE CONSTRAINT unique_form IF NOT EXISTS FOR (n:Form) REQUIRE n.formId IS UNIQUE')


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x116f12490>, keys=[])

In [17]:
# Cypher query and helper function for creating form nodes
merge_form_node_query = """
MERGE (f:Form {formId: $formInfo.formId })
  ON CREATE 
    SET f.names = $formInfo.names
    SET f.source = $formInfo.source
    SET f.cik = $formInfo.ci
    SET f.cusip6 = $formInfo.cusip6
RETURN f.formId
"""

# Helper function to create nodes for all chunks.
# This will use the `merge_chunk_node_query` to create a `:Chunk` node for each chunk.
def create_form_node(form_with_metadata):
  print(f"Creating `:Form` node for form ID {form_with_metadata['formId']}")
  gdb.execute_query(merge_form_node_query, 
          formInfo=form_with_metadata
          )

In [18]:
%%time

counter = 0

all_forms = []

for file_name in all_json_files:
    counter += 1
    print(f'=== Processing {counter} of {len(all_json_files)} ===')
    # get form data from the files
    print(f'Reading Form10k file {file_name}...')
    form_with_metadata = extract_form10k_form_from_file(file_name)
    all_forms.append(form_with_metadata)
    # create node
    print('Creating Form Node...')
    create_form_node(form_with_metadata)
    print(f'Done Processing {file_name}')

# Check the number of nodes in the graph
gdb.execute_query("MATCH (n:Form) RETURN count(n) as formCount").records

=== Processing 1 of 10 ===
Reading Form10k file ../data/sample/form10k/0001650372-23-000040.json...
Creating Form Node...
Creating `:Form` node for form ID 0001650372-23-000040
Done Processing ../data/sample/form10k/0001650372-23-000040.json
=== Processing 2 of 10 ===
Reading Form10k file ../data/sample/form10k/0000950170-23-033201.json...
Creating Form Node...
Creating `:Form` node for form ID 0000950170-23-033201
Done Processing ../data/sample/form10k/0000950170-23-033201.json
=== Processing 3 of 10 ===
Reading Form10k file ../data/sample/form10k/0001564708-23-000368.json...
Creating Form Node...
Creating `:Form` node for form ID 0001564708-23-000368
Done Processing ../data/sample/form10k/0001564708-23-000368.json
=== Processing 4 of 10 ===
Reading Form10k file ../data/sample/form10k/0001558370-23-011516.json...
Creating Form Node...
Creating `:Form` node for form ID 0001558370-23-011516
Done Processing ../data/sample/form10k/0001558370-23-011516.json
=== Processing 5 of 10 ===
Readi

[<Record formCount=10>]

In [19]:
# create a vector index for the form summaries
text_embedding = embeddings_api.embed_query("embed this text using an LLM")
vector_dimensions = len(text_embedding) 

print(f"Text embeddings will have {vector_dimensions} dimensions")
# Create a vector index called "form_10k_forms" the `summaryEmbedding`` property of nodes labeled `Form`. 
gdb.execute_query("""
         CREATE VECTOR INDEX `form_10k_forms` IF NOT EXISTS
          FOR (f:Form) ON (f.summaryEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: $vectorDimensionsParam,
            `vector.similarity_function`: 'cosine'    
         }}
""",
  vectorDimensionsParam=vector_dimensions
)

# Check the vector indexes in the graph
showIndexes()

Text embeddings will have 1536 dimensions
VECTOR INDEX: form_10k_chunks ['Chunk'] ['textEmbedding']
VECTOR INDEX: form_10k_forms ['Form'] ['summaryEmbedding']
RANGE INDEX: unique_chunk ['Chunk'] ['chunkId']
RANGE INDEX: unique_form ['Form'] ['formId']


In [20]:
# Split forms into larger chunks, then summarize each large chunk,
# saving the synthesized summary onto the form node
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 60000,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = False,
)

for form_info in all_forms:
  split_text = text_splitter.split_text(form_info['fullText'])
  summary = ''
  for partial_text in split_text:
    partial_summary = chat_api.invoke(
      f"""Write a single, very brief sentence summary of {form_info['names'][0]}'s business
       based on the following information...\n {partial_text}
      """)
    summary += partial_summary.content + '\n\n'
  print(f"Summarized {form_info['names'][0]}'s form-10k in {len(summary)} characters. Here's a preview...")
  print(f"\t{summary[:120]}")
  form_info['summary'] = summary
  summary_embedding = embeddings_api.embed_query(summary)
  form_info['summaryEmbedding'] = summary_embedding
  print(f"\tUpdating form with ID {form_info['formId']} with summary and embedding...")
  gdb.execute_query("""
    MATCH (f:Form {formId: $formInfoParam.formId})
      SET f.summary = $formInfoParam.summary 
    WITH f
      CALL db.create.setNodeVectorProperty(f, "summaryEmbedding", $formInfoParam.summaryEmbedding)    
    """, 
      formInfoParam=form_info
  )


Summarized ATLASSIAN CORP PLC's form-10k in 1480 characters. Here's a preview...
	ATLASSIAN CORP PLC provides team collaboration software solutions for businesses of all sizes and industries.

ATLASSIAN
	Updating form with ID 0001650372-23-000040 with summary and embedding...
Summarized FedEx Corp's form-10k in 1485 characters. Here's a preview...
	FedEx Corp provides a broad range of transportation, e-commerce, and business services through its operating companies F
	Updating form with ID 0000950170-23-033201 with summary and embedding...
Summarized NEWS CORP   CLASS B's form-10k in 8343 characters. Here's a preview...
	News Corp is a global diversified media and information services company with a focus on creating and distributing autho
	Updating form with ID 0001564708-23-000368 with summary and embedding...
Summarized GSI TECHNOLOGY INC's form-10k in 858 characters. Here's a preview...
	GSI TECHNOLOGY INC provides in-place associative computing solutions for high-growth markets su

In [21]:
# Connect all chunks to their parent `Form` node...

# MATCH a paired node pattern, for the `Chunk` and `Form` nodes
# WHERE the `Chunk` and `Form` nodes have the same `formId` property
# connect the pairs into a (:Chunk)-[:PART_OF]->(:Form) relationship
cypher = """
  MATCH (c:Chunk), (f:Form)
    WHERE c.formId = f.formId
  MERGE (c)-[newRelationship:PART_OF]->(f)
  RETURN count(newRelationship)
"""

gdb.execute_query(cypher).records

[<Record count(newRelationship)=544>]

In [22]:
# Connect all parent `Form` nodes to the "head" of each section linked list...

# MATCH a paired node pattern, for the `Chunk` and `Form` nodes
# WHERE the `Chunk` and `Form` nodes have the same `formId` property
# (this is exactly like a JOIN in SQL)
# connect the pairs with a (:Chunk)-[:PART_OF]->(:Form) relationship
cypher = """
  MATCH (headOfSection:Chunk), (f:Form)
  WHERE headOfSection.formId = f.formId
    AND headOfSection.chunkSeqId = 0
  WITH headOfSection, f
    MERGE (f)-[newRelationship:SECTION {item:headOfSection.item}]->(headOfSection)
  RETURN count(newRelationship)
"""

gdb.execute_query(cypher).records

[<Record count(newRelationship)=40>]

## Add structured data from form-13.csv file

In [23]:
import csv

all_form13s = []

with open(f'{IMPORT_DATA_DIRECTORY}form13.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader: # each row will be a dictionary
      all_form13s.append(row)

first_form13 = all_form13s[0]

first_form13

{'source': 'https://sec.gov/Archives/edgar/data/1000097/0001000097-23-000009.txt',
 'managerCik': '1000097',
 'managerAddress': '152 WEST 57TH STREET, 50TH FLOOR, NEW YORK, NY, 10019',
 'managerName': 'KINGDON CAPITAL MANAGEMENT, L.L.C.',
 'reportCalendarOrQuarter': '2023-06-30',
 'cusip6': '697435',
 'cusip': '697435105',
 'companyName': 'PALO ALTO NETWORKS INC',
 'value': '27595080000.0',
 'shares': '108000'}

In [24]:
# Create a uniqueness constraint on the managerCik property of Manager nodes 
gdb.execute_query("""
    CREATE CONSTRAINT unique_company 
        IF NOT EXISTS FOR (n:Company) 
        REQUIRE n.cusip6 IS UNIQUE
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x12a160f90>, keys=[])

In [25]:
# Slow process of iterating over all rows, transactionally upserting a public company
# Could be batched.
cypher = """
MERGE (com:Company {cusip6: $form13Param.cusip6})
  ON CREATE
    SET com.name = $form13Param.companyName,
        com.cusip = $form13Param.cusip
"""

for form13 in all_form13s:
  gdb.execute_query(cypher, 
      form13Param = form13 
  )


In [26]:
# Create a full-text index of Manager names
gdb.execute_query("""
CREATE FULLTEXT INDEX fullTextCompanyNames
  IF NOT EXISTS
  FOR (com:Company) 
  ON EACH [com.names]
""")


EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x12a83eed0>, keys=[])

In [27]:
# Connect all `Company` nodes to their corresponding `Form` nodes
# based on the `cusip6` property

# MATCH a double node pattern, for the `Company` and `Form` nodes
# WHERE the `Company` and `Form` nodes have the same `cusip6` property
# MERGE to connect these pairs with a (:Company)-[:FILED]->(:Form) relationship
# RETURN a count of the number of relationships created or found (merged)
cypher = """
  MATCH (com:Company), (form:Form)
    WHERE com.cusip6 = form.cusip6
  SET com.names = form.names
  MERGE (com)-[:FILED]->(form)
"""

gdb.execute_query(cypher)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x12a84ee90>, keys=[])

In [28]:
# Create a uniqueness constraint on the managerCik property of Manager nodes 
gdb.execute_query("""
CREATE CONSTRAINT unique_manager 
  IF NOT EXISTS
  FOR (n:Manager) 
  REQUIRE n.cik IS UNIQUE
""")
# Create a full-text index of Manager names
gdb.execute_query("""
CREATE FULLTEXT INDEX fullTextManagerNames
  IF NOT EXISTS
  FOR (mgr:Manager) 
  ON EACH [mgr.name]
""")
showIndexes()

VECTOR INDEX: form_10k_chunks ['Chunk'] ['textEmbedding']
VECTOR INDEX: form_10k_forms ['Form'] ['summaryEmbedding']
FULLTEXT INDEX: fullTextCompanyNames ['Company'] ['names']
FULLTEXT INDEX: fullTextManagerNames ['Manager'] ['name']
RANGE INDEX: unique_chunk ['Chunk'] ['chunkId']
RANGE INDEX: unique_company ['Company'] ['cusip6']
RANGE INDEX: unique_form ['Form'] ['formId']
RANGE INDEX: unique_manager ['Manager'] ['cik']


In [30]:
cypher = """
  MERGE (mgr:Manager {cik: $managerParam.managerCik})
    ON CREATE
        SET mgr.name = $managerParam.managerName,
            mgr.address = $managerParam.managerAddress
"""

for form13 in tqdm(all_form13s):
  gdb.execute_query(cypher, managerParam=form13)


100%|██████████| 6172/6172 [00:20<00:00, 303.18it/s]


In [31]:
# This could all be collapsed into a single pass of the csv
cypher = """
  MATCH (mgr:Manager {cik: $ownsParam.managerCik}), 
        (com:Company {cusip6: $ownsParam.cusip6})
  MERGE (mgr)-[owns:OWNS_STOCK_IN { reportCalendarOrQuarter: $ownsParam.reportCalendarOrQuarter }]->(com)
    ON CREATE
      SET owns.value  = toFloat($ownsParam.value), 
          owns.shares = toInteger($ownsParam.shares)
"""

for form13 in tqdm(all_form13s):
  gdb.execute_query(cypher, ownsParam=form13)


100%|██████████| 6172/6172 [02:28<00:00, 41.66it/s]


## Add geocoding to addresses

In [32]:
import googlemaps

gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)

# Geocoding an address
geocode_result = gmaps.geocode('1600 Amphitheatre Parkway, Mountain View, CA')

geocode_result

[{'address_components': [{'long_name': '1600',
    'short_name': '1600',
    'types': ['street_number']},
   {'long_name': 'Amphitheatre Parkway',
    'short_name': 'Amphitheatre Pkwy',
    'types': ['route']},
   {'long_name': 'Mountain View',
    'short_name': 'Mountain View',
    'types': ['locality', 'political']},
   {'long_name': 'Santa Clara County',
    'short_name': 'Santa Clara County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'California',
    'short_name': 'CA',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '94043', 'short_name': '94043', 'types': ['postal_code']},
   {'long_name': '1351',
    'short_name': '1351',
    'types': ['postal_code_suffix']}],
  'formatted_address': '1600 Amphitheatre Pkwy, Mountain View, CA 94043, USA',
  'geometry': {'location': {'lat': 37.4213908, 'lng': -122.0841182},
   'location_

In [33]:
def filter_city(address_component):
    if (address_component['types'] == ['locality', 'political']
        or address_component['types'] == ['administrative_area_level_3', 'political'] 
        or address_component['types'] == ['postal_town']
        or address_component['types'] == ['political', 'sublocality', 'sublocality_level_1']
        or address_component['types'] == ['neighborhood', 'political']
        or address_component['types'] == ['locality', 'political']
        ):
          return True
    return False

def get_city(geocode):
    found_city = None
    administrative_area_level_3 = None
    locality = None
    postal_town = None
    sublocality_level_1 = None
    neighborhood = None

    for address_component in geocode['address_components']:
        match address_component['types']:
            case ['administrative_area_level_3', 'political']:
                administrative_area_level_3 = address_component
                continue
            case ['locality', 'political']:
                locality = address_component
                continue
            case ['postal_town']:
                postal_town = address_component
                continue
            case ['political', 'sublocality', 'sublocality_level_1']:
                sublocality_level_1 = address_component
                continue
            case ['neighborhood', 'political']:
                neighborhood = address_component
                continue
    possible_city = [locality, administrative_area_level_3, sublocality_level_1, postal_town, neighborhood]
    found_city = list(filter(None, possible_city))
    return found_city[0] if found_city else None

def filter_state(address_component):
    if address_component['types'] == ['administrative_area_level_1', 'political']:
          return True  
    return False

def get_state(geocode):
    state_list = list(filter(filter_state, geocode['address_components']))
    if (state_list):
        return state_list[0]
    else:
        return None
    
def filter_postalcode(address_component):
    if address_component['types'] == ['postal_code']:
          return True  
    return False
    
def get_postalcode(geocode):
    zipcode_list = list(filter(filter_postalcode, geocode['address_components']))
    if (zipcode_list):
        return zipcode_list[0]
    else:
        return None
    
def filter_country(address_component):
    if address_component['types'] == ['country', 'political']:
          return True  
    return False

def get_country(geocode):
    country_list = list(filter(filter_country, geocode['address_components']))
    if (country_list):
        return country_list[0]
    else:
        return None

def get_location(geocode):
    return geocode['geometry']['location']


def long_name(address_component):
    if address_component:
        return address_component['long_name'] if address_component['long_name'] else None
    else:
        return None

def print_address(name, address, city, state, postal, country):
    print(f"{name} is located at {address}")
    print(f"\tcomponents: {city}, {state} {postal}, {country}")
    


In [34]:
gdb.execute_query("""
  CREATE POINT INDEX address_locations IF NOT EXISTS
  FOR (n:Address) ON (n.location)
""")

showIndexes()

POINT INDEX: address_locations ['Address'] ['location']
VECTOR INDEX: form_10k_chunks ['Chunk'] ['textEmbedding']
VECTOR INDEX: form_10k_forms ['Form'] ['summaryEmbedding']
FULLTEXT INDEX: fullTextCompanyNames ['Company'] ['names']
FULLTEXT INDEX: fullTextManagerNames ['Manager'] ['name']
RANGE INDEX: unique_chunk ['Chunk'] ['chunkId']
RANGE INDEX: unique_company ['Company'] ['cusip6']
RANGE INDEX: unique_form ['Form'] ['formId']
RANGE INDEX: unique_manager ['Manager'] ['cik']


In [35]:
get_managers_cypher = """
  MATCH (mgr:Manager)
  RETURN mgr { .cik, .name, .address, .location}
"""

manager_rows = gdb.execute_query(get_managers_cypher).records

managers = list(map(lambda row: row['mgr'], manager_rows))

In [36]:
for manager in tqdm(managers):
  if 'geocode' not in manager:
    geocode_for_address = gmaps.geocode(manager['address'])
    if len(geocode_for_address) > 0:
      manager['geocode'] = geocode_for_address[0] # accept first result


100%|██████████| 2738/2738 [05:12<00:00,  8.77it/s]


In [37]:
gdb.execute_query("""
CREATE INDEX composite_address_index IF NOT EXISTS
FOR (n:Address) 
ON (n.city, n.state)
""")

set_manager_location_cypher = """
  MATCH (mgr:Manager {cik: $managerCik})
  SET mgr.location = point({latitude: $latitude, longitude: $longitude})
  MERGE (addr:Address {city: $city, state: $state})
   ON CREATE SET addr.country = $country
   ON MATCH SET addr.location = point({latitude: $latitude, longitude: $longitude})
  MERGE (mgr)-[:LOCATED_AT]->(addr)
"""

for manager in managers:
  if 'geocode' not in manager:
    continue

  location = get_location(manager['geocode'])
  city = get_city(manager['geocode'])
  state = get_state(manager['geocode'])
  country = get_country(manager['geocode'])
  postal = get_postalcode(manager['geocode'])

  cityOrState = city if city else state
  stateOrCountry = state if state else country 

  print_address(manager['name'], manager['address'],
      long_name(cityOrState), 
      long_name(stateOrCountry), 
      long_name(postal), 
      long_name(country)
  )

  if location and cityOrState and stateOrCountry:
    gdb.execute_query(set_manager_location_cypher,
      managerCik= manager['cik'],
      latitude= location['lat'],
      longitude= location['lng'],
      city= long_name(cityOrState),
      state= long_name(stateOrCountry),
      country= long_name(country)
    )
  else:
    print (f"no location for {manager['name']} {manager['cik']} within {manager['geocode']}")


KINGDON CAPITAL MANAGEMENT, L.L.C. is located at 152 WEST 57TH STREET, 50TH FLOOR, NEW YORK, NY, 10019
	components: New York, New York 10019, United States
Royal Bank of Canada is located at ROYAL BANK PLAZA, 200 BAY STREET, TORONTO, A6, M5J2J5
	components: Toronto, Ontario M5J 2T6, Canada
BROOKFIELD Corp /ON/ is located at BROOKFIELD PLACE, 181 BAY ST, STE 100, PO BOX 762, TORONTO, A6, M5J2T3
	components: Toronto, Ontario M5J 2T3, Canada
COMPASS CAPITAL MANAGEMENT, INC is located at 706 SECOND AVENUE SOUTH, SUITE 400, MINNEAPOLIS, MN, 55402
	components: Minneapolis, Minnesota 55402, United States
SHELTON CAPITAL MANAGEMENT is located at 1875 Lawrence Street, Suite 300, Denver, CO, 80202-1805
	components: Denver, Colorado 80202, United States
AGF MANAGEMENT LTD is located at CIBC SQUARE, TOWER ONE, 81 BAY STREET, SUITE 3900, TORONTO, A6, M5J 0G1
	components: Toronto, Ontario M5J 1E6, Canada
VIRGINIA RETIREMENT SYSTEMS ET AL is located at P.O. BOX 2500, RICHMOND, VA, 23218
	components: 

In [38]:
gdb.execute_query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames", "Blackrock") YIELD node, score
  MATCH p=(node)-[:LOCATED_AT]->(address:Address)
  RETURN p
""").records

[<Record p=<Path start=<Node element_id='4:fb32f79c-b135-44db-b145-e9e041559db3:1041' labels=frozenset({'Manager'}) properties={'address': '50 Hudson Yards, New York, NY, 10001', 'cik': '1364742', 'name': 'BlackRock Inc.', 'location': POINT(-74.0005774 40.7547224)}> end=<Node element_id='4:fb32f79c-b135-44db-b145-e9e041559db3:3302' labels=frozenset({'Address'}) properties={'country': 'United States', 'city': 'New York', 'location': POINT(-73.97462999999999 40.7602617), 'state': 'New York'}> size=1>>]

In [40]:
# How many investment firms are in the same city as Blackrock?
gdb.execute_query("""
  CALL db.index.fulltext.queryNodes("fullTextManagerNames", "Blackrock") YIELD node, score
  MATCH p=(node)-[:LOCATED_AT]->(address:Address)<-[:LOCATED_AT]-(other:Manager)
  RETURN count(other) as numManagers
""").records

[<Record numManagers=223>]

In [41]:
from langchain_community.vectorstores import Neo4jVector
from langchain.chains import RetrievalQAWithSourcesChain

company_rows = kg.query("""
  MATCH (com:Company)
  RETURN com { .cusip6, .name } as company
""")

companies = list(map(lambda row: row['company'], company_rows))

# Create a langchain vector store from the existing Neo4j knowledge graph.
neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

# Create a retriever from the vector store
retriever = neo4j_vector_store.as_retriever()

# Create a chatbot Question Answering chain from the retriever
geo_chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0), chain_type="stuff", retriever=retriever
)


In [42]:
set_company_location_cypher = """
  MATCH (com:Company {cusip6: $cusip6})
  SET com.address = $address
  SET com.location = point({latitude: $latitude, longitude: $longitude})
  MERGE (addr:Address {city: $city, state: $state} )
   ON CREATE SET addr.country = $country
   ON MATCH SET addr.location = point({latitude: $latitude, longitude: $longitude})
  MERGE (com)-[:LOCATED_AT]->(addr)
"""

for company in companies:
    result = geo_chain(f"Where is {company['name']} headquartered?")
    address_statement = result['answer']
    address_geocodes = gmaps.geocode(address_statement)
    if len(address_geocodes) > 0:
        address_geocode = address_geocodes[0]

        cusip6 = company['cusip6']

        location = get_location(address_geocode)
        city = get_city(address_geocode)
        state = get_state(address_geocode)
        postal = get_postalcode(address_geocode)
        country = get_country(address_geocode)

        cityOrState = city if city else state
        stateOrCountry = state if state else country 

        print_address(company['name'], address_geocode['formatted_address'],
            long_name(cityOrState), 
            long_name(stateOrCountry), 
            long_name(postal), 
            long_name(country)
        )

        if location and cityOrState and stateOrCountry:
            kg.query(set_company_location_cypher, params={
                "address": address_geocode['formatted_address'],
                "cusip6": company['cusip6'],
                "latitude": location['lat'],
                "longitude": location['lng'],
                "city": long_name(cityOrState),
                "state": long_name(stateOrCountry),
                "country": long_name(country)
            })
    else:
        print(f"no geocode found for {company['name']} at {address_statement}")

  0%|          | 0/10 [00:00<?, ?it/s]/var/folders/t_/xcdyhzm5185g8bsc5klh4y6h0000gn/T/ipykernel_25273/2640302779.py:12: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = geo_chain(f"Where is {company['name']} headquartered?")
 10%|█         | 1/10 [00:02<00:21,  2.35s/it]

PALO ALTO NETWORKS INC is located at 3000 Tannery Way, Santa Clara, CA 95054, USA
	components: Santa Clara, California 95054, United States


 20%|██        | 2/10 [00:04<00:18,  2.33s/it]

FEDEX CORP is located at Delaware, USA
	components: Delaware, Delaware None, United States


 30%|███       | 3/10 [00:06<00:14,  2.03s/it]

...

SEAGATE TECHNOLOGY is located at 2445 Augustine Dr, Santa Clara, CA 95054, USA
	components: Santa Clara, California 95054, United States


 90%|█████████ | 9/10 [00:18<00:02,  2.12s/it]

ATLASSIAN CORP PLC is located at Santa Clara, CA, USA
	components: Santa Clara, California None, United States


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]

APPLE INC is located at Cupertino, CA, USA
	components: Cupertino, California None, United States
